In [2]:
import prody.proteins.stride
from prody import *
import numpy as np
import os
from biopandas.pdb import PandasPdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import Bio
from Bio.PDB import PDBParser
from scipy.spatial.distance import euclidean
import itertools
import utility
import matplotlib.gridspec as gridspec


In [3]:
class SeabornFig2Grid():

    def __init__(self, seaborngrid, fig,  subplot_spec):
        self.fig = fig
        self.sg = seaborngrid
        self.subplot = subplot_spec
        if isinstance(self.sg, sns.axisgrid.FacetGrid) or \
            isinstance(self.sg, sns.axisgrid.PairGrid):
            self._movegrid()
        elif isinstance(self.sg, sns.axisgrid.JointGrid):
            self._movejointgrid()
        self._finalize()

    def _movegrid(self):
        """ Move PairGrid or Facetgrid """
        self._resize()
        n = self.sg.axes.shape[0]
        m = self.sg.axes.shape[1]
        self.subgrid = gridspec.GridSpecFromSubplotSpec(n,m, subplot_spec=self.subplot)
        for i in range(n):
            for j in range(m):
                self._moveaxes(self.sg.axes[i,j], self.subgrid[i,j])

    def _movejointgrid(self):
        """ Move Jointgrid """
        h= self.sg.ax_joint.get_position().height
        h2= self.sg.ax_marg_x.get_position().height
        r = int(np.round(h/h2))
        self._resize()
        self.subgrid = gridspec.GridSpecFromSubplotSpec(r+1,r+1, subplot_spec=self.subplot)

        self._moveaxes(self.sg.ax_joint, self.subgrid[1:, :-1])
        self._moveaxes(self.sg.ax_marg_x, self.subgrid[0, :-1])
        self._moveaxes(self.sg.ax_marg_y, self.subgrid[1:, -1])

    def _moveaxes(self, ax, gs):
        #https://stackoverflow.com/a/46906599/4124317
        ax.remove()
        ax.figure=self.fig
        self.fig.axes.append(ax)
        self.fig.add_axes(ax)
        ax._subplotspec = gs
        ax.set_position(gs.get_position(self.fig))
        ax.set_subplotspec(gs)

    def _finalize(self):
        plt.close(self.sg.fig)
        self.fig.canvas.mpl_connect("resize_event", self._resize)
        self.fig.canvas.draw()

    def _resize(self, evt=None):
        self.sg.fig.set_size_inches(self.fig.get_size_inches())

In [5]:
data_dir = '../../data'
processed_data_dir = os.path.join(data_dir, 'processed_data_training')
result_dir = '../../results'

In [6]:
feature_all = np.load(os.path.join(processed_data_dir, 'feature_all.npy'))

In [7]:
data = pd.DataFrame(data=feature_all, columns=['c1', 'c1_pos', 'c1_neg', 'c1_polar', 'c1_amp', 'c1_hp', 'c1_hp_idx',
                                       'c1_rd', 'c1_shell', 'c1_poc', 
                                       'c1_N_count', 'c1_C_count', 'c1_O_count','c1_H_count', 'c1_S_count', 
                                       'c1_N_charge', 'c1_C_charge', 'c1_O_charge','c1_H_charge', 'c1_S_charge', 
                                       'c1_ollivier', 'c1_forman', 'c1_gnm',
                                       'fd1', 'more_fd1_1', 'more_fd1_2', 'more_fd1_3', 'more_fd1_4',
                                       'G_os1_5', 'G_os1_7', 'G_os1_10', 'G_os1_15', 
                                       'c2', 'c2_pos', 'c2_neg','c2_polar', 'c2_amp', 'c2_hp', 'c2_hp_idx', 
                                       'c2_rd', 'c2_shell','c2_poc', 
                                       'c2_N_count', 'c2_C_count', 'c2_O_count', 'c2_H_count','c2_S_count', 
                                       'c2_N_charge', 'c2_C_charge', 'c2_O_charge','c2_H_charge', 'c2_S_charge', 
                                       'c2_ollivier', 'c2_forman', 'c2_gnm',
                                       'fd2', 'more_fd2_1', 'more_fd2_2', 'more_fd2_3', 'more_fd2_4',
                                       'G_os2_5', 'G_os2_7', 'G_os2_10', 'G_os2_15', 
                                       'distance'])


In [8]:
d0 = data[data['distance']==0]
d1 = data[data['distance']==1].sample(n=len(d0))
d2 = data[data['distance']==2].sample(n=len(d0))
d3 = data[data['distance']==3].sample(n=len(d0))
d4 = data[data['distance']==4].sample(n=len(d0))


# chem : pink, violet, plum

In [10]:
with sns.axes_style("white"):
    g0 = sns.jointplot(x=d0['c1'], y=d0['c2'], kind='kde', color='pink')
    #g0.set_xticklabels(labels)
    g1 = sns.jointplot(x=d1['c1'], y=d1['c2'], kind='kde', color='pink')
    g2 = sns.jointplot(x=d2['c1'], y=d2['c2'], kind='kde', color='pink')
    g3 = sns.jointplot(x=d3['c1'], y=d3['c2'], kind='kde', color='pink')
    g4 = sns.jointplot(x=d4['c1'], y=d4['c2'], kind='kde', color='pink')
    
    fig = plt.figure(figsize=(20,4))
    gs = gridspec.GridSpec(1,5)
    
    mg0 = SeabornFig2Grid(g0, fig, gs[0])
    mg1 = SeabornFig2Grid(g1, fig, gs[1])
    mg2 = SeabornFig2Grid(g2, fig, gs[2])
    mg3 = SeabornFig2Grid(g3, fig, gs[3])
    mg4 = SeabornFig2Grid(g4, fig, gs[4])
    
    gs.tight_layout(fig)
    plt.savefig(os.path.join(result_dir, 'AA_TYPE_stat.png'))
    plt.show()
    

In [ ]:
with sns.axes_style("white"):
    g0 = sns.jointplot(x=d0['c1_hp_idx'], y=d0['c2_hp_idx'], kind='hex', color='violet')
    g1 = sns.jointplot(x=d1['c1_hp_idx'], y=d1['c2_hp_idx'], kind='hex', color='violet')
    g2 = sns.jointplot(x=d2['c1_hp_idx'], y=d2['c2_hp_idx'], kind='hex', color='violet')
    g3 = sns.jointplot(x=d3['c1_hp_idx'], y=d3['c2_hp_idx'], kind='hex', color='violet')
    g4 = sns.jointplot(x=d4['c1_hp_idx'], y=d4['c2_hp_idx'], kind='hex', color='violet')
    
    fig = plt.figure(figsize=(20,4))
    gs = gridspec.GridSpec(1,5)
    
    mg0 = SeabornFig2Grid(g0, fig, gs[0])
    mg1 = SeabornFig2Grid(g1, fig, gs[1])
    mg2 = SeabornFig2Grid(g2, fig, gs[2])
    mg3 = SeabornFig2Grid(g3, fig, gs[3])
    mg4 = SeabornFig2Grid(g4, fig, gs[4])
    
    gs.tight_layout(fig)
    plt.savefig(os.path.join(result_dir, 'HP_stat.png'))
    plt.show()
    

In [ ]:
with sns.axes_style("white"):
    g0 = sns.jointplot(x=d0['c1_C_count'], y=d0['c2_C_count'], kind='kde', color='plum')
    g1 = sns.jointplot(x=d1['c1_C_count'], y=d1['c2_C_count'], kind='kde', color='plum')
    g2 = sns.jointplot(x=d2['c1_C_count'], y=d2['c2_C_count'], kind='kde', color='plum')
    g3 = sns.jointplot(x=d3['c1_C_count'], y=d3['c2_C_count'], kind='kde', color='plum')
    g4 = sns.jointplot(x=d4['c1_C_count'], y=d4['c2_C_count'], kind='kde', color='plum')
    
    fig = plt.figure(figsize=(20,4))
    gs = gridspec.GridSpec(1,5)
    
    mg0 = SeabornFig2Grid(g0, fig, gs[0])
    mg1 = SeabornFig2Grid(g1, fig, gs[1])
    mg2 = SeabornFig2Grid(g2, fig, gs[2])
    mg3 = SeabornFig2Grid(g3, fig, gs[3])
    mg4 = SeabornFig2Grid(g4, fig, gs[4])
    
    gs.tight_layout(fig)
    plt.savefig(os.path.join(result_dir, 'C_count_stat.png'))
    plt.show()

In [ ]:
with sns.axes_style("white"):
    g0 = sns.jointplot(x=d0['c1_C_charge'], y=d0['c2_C_charge'], kind='kde', color='plum')
    g1 = sns.jointplot(x=d1['c1_C_charge'], y=d1['c2_C_charge'], kind='kde', color='plum')
    g2 = sns.jointplot(x=d2['c1_C_charge'], y=d2['c2_C_charge'], kind='kde', color='plum')
    g3 = sns.jointplot(x=d3['c1_C_charge'], y=d3['c2_C_charge'], kind='kde', color='plum')
    g4 = sns.jointplot(x=d4['c1_C_charge'], y=d4['c2_C_charge'], kind='kde', color='plum')
    
    fig = plt.figure(figsize=(20,4))
    gs = gridspec.GridSpec(1,5)
    
    mg0 = SeabornFig2Grid(g0, fig, gs[0])
    mg1 = SeabornFig2Grid(g1, fig, gs[1])
    mg2 = SeabornFig2Grid(g2, fig, gs[2])
    mg3 = SeabornFig2Grid(g3, fig, gs[3])
    mg4 = SeabornFig2Grid(g4, fig, gs[4])
    
    gs.tight_layout(fig)
    plt.savefig(os.path.join(result_dir, 'C_charge_stat.png'))
    plt.show()

In [ ]:
# with sns.axes_style("white"):
#     g0 = sns.jointplot(x=d0['c1_N_count'], y=d0['c2_N_count'], kind='kde', color='plum')
#     g1 = sns.jointplot(x=d1['c1_N_count'], y=d1['c2_N_count'], kind='kde', color='plum')
#     g2 = sns.jointplot(x=d2['c1_N_count'], y=d2['c2_N_count'], kind='kde', color='plum')
#     g3 = sns.jointplot(x=d3['c1_N_count'], y=d3['c2_N_count'], kind='kde', color='plum')
#     g4 = sns.jointplot(x=d4['c1_N_count'], y=d4['c2_N_count'], kind='kde', color='plum')
    
#     fig = plt.figure(figsize=(20,4))
#     gs = gridspec.GridSpec(1,5)
    
#     mg0 = SeabornFig2Grid(g0, fig, gs[0])
#     mg1 = SeabornFig2Grid(g1, fig, gs[1])
#     mg2 = SeabornFig2Grid(g2, fig, gs[2])
#     mg3 = SeabornFig2Grid(g3, fig, gs[3])
#     mg4 = SeabornFig2Grid(g4, fig, gs[4])
    
#     gs.tight_layout(fig)
#     plt.savefig(os.path.join(feature_stat, 'N_count.png'))
#     plt.show()

In [ ]:
# with sns.axes_style("white"):
#     g0 = sns.jointplot(x=d0['c1_N_charge'], y=d0['c2_N_charge'], kind='kde', color='plum')
#     g1 = sns.jointplot(x=d1['c1_N_charge'], y=d1['c2_N_charge'], kind='kde', color='plum')
#     g2 = sns.jointplot(x=d2['c1_N_charge'], y=d2['c2_N_charge'], kind='kde', color='plum')
#     g3 = sns.jointplot(x=d3['c1_N_charge'], y=d3['c2_N_charge'], kind='kde', color='plum')
#     g4 = sns.jointplot(x=d4['c1_N_charge'], y=d4['c2_N_charge'], kind='kde', color='plum')
    
#     fig = plt.figure(figsize=(20,4))
#     gs = gridspec.GridSpec(1,5)
    
#     mg0 = SeabornFig2Grid(g0, fig, gs[0])
#     mg1 = SeabornFig2Grid(g1, fig, gs[1])
#     mg2 = SeabornFig2Grid(g2, fig, gs[2])
#     mg3 = SeabornFig2Grid(g3, fig, gs[3])
#     mg4 = SeabornFig2Grid(g4, fig, gs[4])
    
#     gs.tight_layout(fig)
#     plt.savefig(os.path.join(feature_stat, 'N_charge.png'))
#     plt.show()

In [ ]:
with sns.axes_style("white"):
    g0 = sns.jointplot(x=d0['c1_H_count'], y=d0['c2_H_count'], kind='kde', color='plum')
    g1 = sns.jointplot(x=d1['c1_H_count'], y=d1['c2_H_count'], kind='kde', color='plum')
    g2 = sns.jointplot(x=d2['c1_H_count'], y=d2['c2_H_count'], kind='kde', color='plum')
    g3 = sns.jointplot(x=d3['c1_H_count'], y=d3['c2_H_count'], kind='kde', color='plum')
    g4 = sns.jointplot(x=d4['c1_H_count'], y=d4['c2_H_count'], kind='kde', color='plum')
    
    fig = plt.figure(figsize=(20,4))
    gs = gridspec.GridSpec(1,5)
    
    mg0 = SeabornFig2Grid(g0, fig, gs[0])
    mg1 = SeabornFig2Grid(g1, fig, gs[1])
    mg2 = SeabornFig2Grid(g2, fig, gs[2])
    mg3 = SeabornFig2Grid(g3, fig, gs[3])
    mg4 = SeabornFig2Grid(g4, fig, gs[4])
    
    gs.tight_layout(fig)
#     plt.savefig(os.path.join(feature_stat, 'H_count.png'))
    plt.show()

In [ ]:
with sns.axes_style("white"):
    g0 = sns.jointplot(x=d0['c1_H_charge'], y=d0['c2_H_charge'], kind='kde', color='plum')
    g1 = sns.jointplot(x=d1['c1_H_charge'], y=d1['c2_H_charge'], kind='kde', color='plum')
    g2 = sns.jointplot(x=d2['c1_H_charge'], y=d2['c2_H_charge'], kind='kde', color='plum')
    g3 = sns.jointplot(x=d3['c1_H_charge'], y=d3['c2_H_charge'], kind='kde', color='plum')
    g4 = sns.jointplot(x=d4['c1_H_charge'], y=d4['c2_H_charge'], kind='kde', color='plum')
    
    fig = plt.figure(figsize=(20,4))
    gs = gridspec.GridSpec(1,5)
    
    mg0 = SeabornFig2Grid(g0, fig, gs[0])
    mg1 = SeabornFig2Grid(g1, fig, gs[1])
    mg2 = SeabornFig2Grid(g2, fig, gs[2])
    mg3 = SeabornFig2Grid(g3, fig, gs[3])
    mg4 = SeabornFig2Grid(g4, fig, gs[4])
    
    gs.tight_layout(fig)
#     plt.savefig(os.path.join(feature_stat, 'H_charge.png'))
    plt.show()

In [ ]:
# with sns.axes_style("white"):
#     g0 = sns.jointplot(x=d0['c1_S_count'], y=d0['c2_S_count'], kind='kde', color='plum')
#     g1 = sns.jointplot(x=d1['c1_S_count'], y=d1['c2_S_count'], kind='kde', color='plum')
#     g2 = sns.jointplot(x=d2['c1_S_count'], y=d2['c2_S_count'], kind='kde', color='plum')
#     g3 = sns.jointplot(x=d3['c1_S_count'], y=d3['c2_S_count'], kind='kde', color='plum')
#     g4 = sns.jointplot(x=d4['c1_S_count'], y=d4['c2_S_count'], kind='kde', color='plum')
    
#     fig = plt.figure(figsize=(20,4))
#     gs = gridspec.GridSpec(1,5)
    
#     mg0 = SeabornFig2Grid(g0, fig, gs[0])
#     mg1 = SeabornFig2Grid(g1, fig, gs[1])
#     mg2 = SeabornFig2Grid(g2, fig, gs[2])
#     mg3 = SeabornFig2Grid(g3, fig, gs[3])
#     mg4 = SeabornFig2Grid(g4, fig, gs[4])
    
#     gs.tight_layout(fig)
#     plt.savefig(os.path.join(feature_stat, 'S_count.png'))
#     plt.show()

In [ ]:
# with sns.axes_style("white"):
#     g0 = sns.jointplot(x=d0['c1_S_charge'], y=d0['c2_S_charge'], kind='kde', color='plum')
#     g1 = sns.jointplot(x=d1['c1_S_charge'], y=d1['c2_S_charge'], kind='kde', color='plum')
#     g2 = sns.jointplot(x=d2['c1_S_charge'], y=d2['c2_S_charge'], kind='kde', color='plum')
#     g3 = sns.jointplot(x=d3['c1_S_charge'], y=d3['c2_S_charge'], kind='kde', color='plum')
#     g4 = sns.jointplot(x=d4['c1_S_charge'], y=d4['c2_S_charge'], kind='kde', color='plum')
    
#     fig = plt.figure(figsize=(20,4))
#     gs = gridspec.GridSpec(1,5)
    
#     mg0 = SeabornFig2Grid(g0, fig, gs[0])
#     mg1 = SeabornFig2Grid(g1, fig, gs[1])
#     mg2 = SeabornFig2Grid(g2, fig, gs[2])
#     mg3 = SeabornFig2Grid(g3, fig, gs[3])
#     mg4 = SeabornFig2Grid(g4, fig, gs[4])
    
#     gs.tight_layout(fig)
#     plt.savefig(os.path.join(feature_stat, 'S_charge.png'))
#     plt.show()

# graph curvature : skyblue

In [ ]:
with sns.axes_style("white"):
    g0 = sns.jointplot(x=d0['c1_ollivier'], y=d0['c2_ollivier'], kind='kde', color='skyblue', xlim={-2,3}, ylim={-2,3})
    g1 = sns.jointplot(x=d1['c1_ollivier'], y=d1['c2_ollivier'], kind='kde', color='skyblue', xlim={-2,3}, ylim={-2,3})
    g2 = sns.jointplot(x=d2['c1_ollivier'], y=d2['c2_ollivier'], kind='kde', color='skyblue', xlim={-2,3}, ylim={-2,3})
    g3 = sns.jointplot(x=d3['c1_ollivier'], y=d3['c2_ollivier'], kind='kde', color='skyblue', xlim={-2,3}, ylim={-2,3})
    g4 = sns.jointplot(x=d4['c1_ollivier'], y=d4['c2_ollivier'], kind='kde', color='skyblue', xlim={-2,3}, ylim={-2,3})
    
    g0.fig.axes[0].invert_yaxis()
    g0.fig.axes[0].invert_xaxis()
    
    g1.fig.axes[0].invert_yaxis()
    g1.fig.axes[0].invert_xaxis()
    
    g2.fig.axes[0].invert_yaxis()
    g2.fig.axes[0].invert_xaxis()
    
    g3.fig.axes[0].invert_yaxis()
    g3.fig.axes[0].invert_xaxis()
    
    g4.fig.axes[0].invert_yaxis()
    g4.fig.axes[0].invert_xaxis()
    
    fig = plt.figure(figsize=(20,4))
    gs = gridspec.GridSpec(1,5)
    
    
    mg0 = SeabornFig2Grid(g0, fig, gs[0])
    mg1 = SeabornFig2Grid(g1, fig, gs[1])
    mg2 = SeabornFig2Grid(g2, fig, gs[2])
    mg3 = SeabornFig2Grid(g3, fig, gs[3])
    mg4 = SeabornFig2Grid(g4, fig, gs[4])
    
    
    
    
    gs.tight_layout(fig)
    plt.savefig(os.path.join(result_dir, 'ollivier_stat.png'))
    plt.show()

In [ ]:
with sns.axes_style("white"):
    g0 = sns.jointplot(x=d0['c1_forman'], y=d0['c2_forman'], kind='kde', color='skyblue', xlim={-80,30}, ylim={-80,30})
    g1 = sns.jointplot(x=d1['c1_forman'], y=d1['c2_forman'], kind='kde', color='skyblue', xlim={-80,30}, ylim={-80,30})
    g2 = sns.jointplot(x=d2['c1_forman'], y=d2['c2_forman'], kind='kde', color='skyblue', xlim={-80,30}, ylim={-80,30})
    g3 = sns.jointplot(x=d3['c1_forman'], y=d3['c2_forman'], kind='kde', color='skyblue', xlim={-80,30}, ylim={-80,30})
    g4 = sns.jointplot(x=d4['c1_forman'], y=d4['c2_forman'], kind='kde', color='skyblue', xlim={-80,30}, ylim={-80,30})
    
    fig = plt.figure(figsize=(20,4))
    gs = gridspec.GridSpec(1,5)
    
    mg0 = SeabornFig2Grid(g0, fig, gs[0])
    mg1 = SeabornFig2Grid(g1, fig, gs[1])
    mg2 = SeabornFig2Grid(g2, fig, gs[2])
    mg3 = SeabornFig2Grid(g3, fig, gs[3])
    mg4 = SeabornFig2Grid(g4, fig, gs[4])
    
    gs.tight_layout(fig)
    plt.savefig(os.path.join(result_dir, 'forman_stat.png'))
    plt.show()

# graph flexible rididity: dodgerblue

In [ ]:
with sns.axes_style("white"):
    g0 = sns.jointplot(x=d0['c1_gnm'], y=d0['c2_gnm'], kind='kde', color='dodgerblue', xlim={0,0.5}, ylim={0,0.5})
    g1 = sns.jointplot(x=d1['c1_gnm'], y=d1['c2_gnm'], kind='kde', color='dodgerblue', xlim={0,0.5}, ylim={0,0.5})
    g2 = sns.jointplot(x=d2['c1_gnm'], y=d2['c2_gnm'], kind='kde', color='dodgerblue', xlim={0,0.5}, ylim={0,0.5})
    g3 = sns.jointplot(x=d3['c1_gnm'], y=d3['c2_gnm'], kind='kde', color='dodgerblue', xlim={0,0.5}, ylim={0,0.5})
    g4 = sns.jointplot(x=d4['c1_gnm'], y=d4['c2_gnm'], kind='kde', color='dodgerblue', xlim={0,0.5}, ylim={0,0.5})
    
    fig = plt.figure(figsize=(20,4))
    gs = gridspec.GridSpec(1,5)
    
    mg0 = SeabornFig2Grid(g0, fig, gs[0])
    mg1 = SeabornFig2Grid(g1, fig, gs[1])
    mg2 = SeabornFig2Grid(g2, fig, gs[2])
    mg3 = SeabornFig2Grid(g3, fig, gs[3])
    mg4 = SeabornFig2Grid(g4, fig, gs[4])
    
    gs.tight_layout(fig)
    plt.savefig(os.path.join(result_dir, 'gnm_stat.png'))
    plt.show()

# graph multifractal: cornflowerblue

In [ ]:
with sns.axes_style("white"):
    g0 = sns.jointplot(x=d0['fd1'], y=d0['fd2'], kind='hex', color='cornflowerblue', xlim={0.3,2.2}, ylim={0.3,2.2})
    g1 = sns.jointplot(x=d1['fd1'], y=d1['fd2'], kind='hex', color='cornflowerblue', xlim={0.3,2.2}, ylim={0.3,2.2})
    g2 = sns.jointplot(x=d2['fd1'], y=d2['fd2'], kind='hex', color='cornflowerblue', xlim={0.3,2.2}, ylim={0.3,2.2})
    g3 = sns.jointplot(x=d3['fd1'], y=d3['fd2'], kind='hex', color='cornflowerblue', xlim={0.3,2.2}, ylim={0.3,2.2})
    g4 = sns.jointplot(x=d4['fd1'], y=d4['fd2'], kind='hex', color='cornflowerblue', xlim={0.3,2.2}, ylim={0.3,2.2})
    
    fig = plt.figure(figsize=(20,4))
    gs = gridspec.GridSpec(1,5)
    
    mg0 = SeabornFig2Grid(g0, fig, gs[0])
    mg1 = SeabornFig2Grid(g1, fig, gs[1])
    mg2 = SeabornFig2Grid(g2, fig, gs[2])
    mg3 = SeabornFig2Grid(g3, fig, gs[3])
    mg4 = SeabornFig2Grid(g4, fig, gs[4])
    
    gs.tight_layout(fig)
    plt.savefig(os.path.join(result_dir, 'fd_stat.png'))
    plt.show()

# graph chiral: mediumaquamarine

In [ ]:
with sns.axes_style("white"):
    g0 = sns.jointplot(x=d0['G_os1_7'], y=d0['G_os2_7'], kind='hex', color='mediumaquamarine', xlim={-1.0e7,1.0e7}, ylim={-1.0e7,1.0e7})
    g1 = sns.jointplot(x=d1['G_os1_7'], y=d1['G_os2_7'], kind='hex', color='mediumaquamarine', xlim={-1.0e7,1.0e7}, ylim={-1.0e7,1.0e7})
    g2 = sns.jointplot(x=d2['G_os1_7'], y=d2['G_os2_7'], kind='hex', color='mediumaquamarine', xlim={-1.0e7,1.0e7}, ylim={-1.0e7,1.0e7})
    g3 = sns.jointplot(x=d3['G_os1_7'], y=d3['G_os2_7'], kind='hex', color='mediumaquamarine', xlim={-1.0e7,1.0e7}, ylim={-1.0e7,1.0e7})
    g4 = sns.jointplot(x=d4['G_os1_7'], y=d4['G_os2_7'], kind='hex', color='mediumaquamarine', xlim={-1.0e7,1.0e7}, ylim={-1.0e7,1.0e7})
    
    fig = plt.figure(figsize=(20,4))
    gs = gridspec.GridSpec(1,5)
    
    mg0 = SeabornFig2Grid(g0, fig, gs[0])
    mg1 = SeabornFig2Grid(g1, fig, gs[1])
    mg2 = SeabornFig2Grid(g2, fig, gs[2])
    mg3 = SeabornFig2Grid(g3, fig, gs[3])
    mg4 = SeabornFig2Grid(g4, fig, gs[4])
    
    gs.tight_layout(fig)
    plt.savefig(os.path.join(result_dir, 'chiral7_stat.png'))
    plt.show()
                 

# pocket geometry : salmon

In [ ]:
with sns.axes_style("white"):
    g0 = sns.jointplot(x=d0['c1_poc'], y=d0['c2_poc'], kind='kde', color='salmon', xlim={-2,28}, ylim={-2,40})
    g1 = sns.jointplot(x=d1['c1_poc'], y=d1['c2_poc'], kind='kde', color='salmon', xlim={-2,28}, ylim={-2,40})
    g2 = sns.jointplot(x=d2['c1_poc'], y=d2['c2_poc'], kind='kde', color='salmon', xlim={-2,28}, ylim={-2,40})
    g3 = sns.jointplot(x=d3['c1_poc'], y=d3['c2_poc'], kind='kde', color='salmon', xlim={-2,28}, ylim={-2,40})
    g4 = sns.jointplot(x=d4['c1_poc'], y=d4['c2_poc'], kind='kde', color='salmon', xlim={-2,28}, ylim={-2,40})
    
    g0.fig.axes[0].invert_yaxis()
    g0.fig.axes[0].invert_xaxis()
    
    g1.fig.axes[0].invert_yaxis()
    g1.fig.axes[0].invert_xaxis()
    
    g2.fig.axes[0].invert_yaxis()
    g2.fig.axes[0].invert_xaxis()
    
    g3.fig.axes[0].invert_yaxis()
    g3.fig.axes[0].invert_xaxis()
    
    g4.fig.axes[0].invert_yaxis()
    g4.fig.axes[0].invert_xaxis()
    
    
    fig = plt.figure(figsize=(20,4))
    gs = gridspec.GridSpec(1,5)
    
    mg0 = SeabornFig2Grid(g0, fig, gs[0])
    mg1 = SeabornFig2Grid(g1, fig, gs[1])
    mg2 = SeabornFig2Grid(g2, fig, gs[2])
    mg3 = SeabornFig2Grid(g3, fig, gs[3])
    mg4 = SeabornFig2Grid(g4, fig, gs[4])
    
    gs.tight_layout(fig)
    plt.savefig(os.path.join(result_dir, 'poc_stat.png'))
    plt.show()

In [ ]:
with sns.axes_style("white"):
    g0 = sns.jointplot(x=d0['c1_rd'], y=d0['c2_rd'], kind='kde', color='salmon', xlim={0,10}, ylim={0,10})
    g1 = sns.jointplot(x=d1['c1_rd'], y=d1['c2_rd'], kind='kde', color='salmon', xlim={0,10}, ylim={0,10})
    g2 = sns.jointplot(x=d2['c1_rd'], y=d2['c2_rd'], kind='kde', color='salmon', xlim={0,10}, ylim={0,10})
    g3 = sns.jointplot(x=d3['c1_rd'], y=d3['c2_rd'], kind='kde', color='salmon', xlim={0,10}, ylim={0,10})
    g4 = sns.jointplot(x=d4['c1_rd'], y=d4['c2_rd'], kind='kde', color='salmon', xlim={0,10}, ylim={0,10})
    
    fig = plt.figure(figsize=(20,4))
    gs = gridspec.GridSpec(1,5)
    
    mg0 = SeabornFig2Grid(g0, fig, gs[0])
    mg1 = SeabornFig2Grid(g1, fig, gs[1])
    mg2 = SeabornFig2Grid(g2, fig, gs[2])
    mg3 = SeabornFig2Grid(g3, fig, gs[3])
    mg4 = SeabornFig2Grid(g4, fig, gs[4])
    
    gs.tight_layout(fig)
    plt.savefig(os.path.join(result_dir, 'rd_stat.png'))
    plt.show()

In [ ]:
with sns.axes_style("white"):
    g0 = sns.jointplot(x=d0['c1_shell'], y=d0['c2_shell'], kind='kde', color='salmon', xlim={20,100}, ylim={20,100})
    g1 = sns.jointplot(x=d1['c1_shell'], y=d1['c2_shell'], kind='kde', color='salmon', xlim={20,100}, ylim={20,100})
    g2 = sns.jointplot(x=d2['c1_shell'], y=d2['c2_shell'], kind='kde', color='salmon', xlim={20,100}, ylim={20,100})
    g3 = sns.jointplot(x=d3['c1_shell'], y=d3['c2_shell'], kind='kde', color='salmon', xlim={20,100}, ylim={20,100})
    g4 = sns.jointplot(x=d4['c1_shell'], y=d4['c2_shell'], kind='kde', color='salmon', xlim={20,100}, ylim={20,100})
    
    fig = plt.figure(figsize=(20,4))
    gs = gridspec.GridSpec(1,5)
    
    mg0 = SeabornFig2Grid(g0, fig, gs[0])
    mg1 = SeabornFig2Grid(g1, fig, gs[1])
    mg2 = SeabornFig2Grid(g2, fig, gs[2])
    mg3 = SeabornFig2Grid(g3, fig, gs[3])
    mg4 = SeabornFig2Grid(g4, fig, gs[4])
    
    gs.tight_layout(fig)
    plt.savefig(os.path.join(result_dir, 'shell_stat.png'))
    plt.show()